In [1]:
from data_gen import *
from models import *
from keras import models

Using TensorFlow backend.


In [2]:

# data generator for gofa word, root word, word feature
dg2 = DataGen(data="data/goffa.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg2.max_root_len
n_steps_out = dg2.max_output_len

In [3]:
model = models.load_model("model.h5")
root_input = model.get_layer("root_word_input").output
feature_input = model.get_layer("word_feature_input").output
target_input = model.get_layer("target_word_input").output

state_h = model.get_layer("concatnate").output
encoder_model = Model([root_input, feature_input], state_h)


decoder_state_input_h = Input(shape=(256,))
decoder_gru = model.get_layer("decoder_gru")
gru_outputs, state_h  = decoder_gru(target_input, initial_state=decoder_state_input_h)

decoder_dense = model.get_layer("train_output")
decoder_outputs = decoder_dense(gru_outputs)

decoder_model = Model([target_input, decoder_state_input_h], [decoder_outputs, state_h])

In [4]:
# infenc - inference encoder model
# infdec - inference decoder model
# train - training model that combines both
# n_input_length - the length of the input and the output
# word_feat_len - the length of the word feature vector
# n_units - size of the hidden memory in the RNN
# model, encoder_model, decoder_model = conv_model(n_input_length, n_input_length, dg2.word_feat_len, 256)
# train.compile(optimizer='adam', loss='categorical_crossentropy')


In [5]:
# number of batches for the gofa data generator
batch_size = 128
n_batches = int(len(dg2.words) * .05 / batch_size) 
gen2 = dg2.cnn_gen_data(batch_size=batch_size, n_batches=n_batches)
print("Train size: ", (n_batches * batch_size))

Train size:  8320


In [6]:
# model train given the data generator, how many batches and number of epochs
history = model.fit_generator(gen2, steps_per_epoch=n_batches, epochs = 8)

Epoch 1/8
65/65 [==============================] - 7s 106ms/step - loss: 0.0945
Epoch 2/8
65/65 [==============================] - 5s 73ms/step - loss: 0.0560
Epoch 3/8
65/65 [==============================] - 5s 73ms/step - loss: 0.0480
Epoch 4/8
65/65 [==============================] - 5s 74ms/step - loss: 0.0385
Epoch 5/8
65/65 [==============================] - 5s 74ms/step - loss: 0.0377
Epoch 6/8
65/65 [==============================] - 5s 76ms/step - loss: 0.0373
Epoch 7/8
65/65 [==============================] - 5s 74ms/step - loss: 0.0372
Epoch 8/8
65/65 [==============================] - 5s 73ms/step - loss: 0.0371


In [7]:
# test data generator for gofa words

g_test_batches = int(len(dg2.words) * .02 / batch_size) 
gen3 = dg2.cnn_gen_data(batch_size=batch_size, n_batches=g_test_batches, trainset=False)
print("Train size: ", (g_test_batches * batch_size))

Train size:  3328


In [8]:

# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(g_test_batches):
    # get data from test data generator
    [X1, X2, X3], y = next(gen3)
    for j in range(batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
        
        # predicts the target word given root word and features
        target = predict(encoder_model, decoder_model, root_word_matrix, word_features, n_steps_out, n_input_length)
        root = ''.join(dg2.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg2.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg2.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal'
#         print(len(dg.one_hot_decode(y[j])), len(dg.one_hot_decode(target)))
#         print(len(dg.one_hot_decode(target)[:27]))
#         print(len(y[j]))
        if dg2.one_hot_decode(y[j]) == dg2.one_hot_decode(target)[:27]:
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
        if root.strip() in targetS.strip():
            in_word += 1
#     print(b, root, word, targetS)
    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 93.09%
